In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# قراءة البيانات
df = pd.read_csv("data/prepro_data.csv")
x = df.drop(columns=['fail'])
y = df['fail']





x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20)

# تعريف النموذج
class SimpleNN(nn.Module):
    def __init__(self, input_dim,hidden_dim,dropout_rate):
        
        super().__init__()
        self.net= nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
     
    def forward(self, x):
        x = self.net(x)
        return x

def train_model(params):
    
    model = SimpleNN(x_train.shape[1], params['hidden_dim'], params['dropout_rate'])
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

# التدريب
    for epoch in range(20):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/20, Loss: {running_loss/len(train_loader):.4f}")
# التقييم
    model.eval()

    with torch.no_grad():
        test_pred = model(x_test_tensor)
        test_loss = loss_fn(test_pred, y_test_tensor)
        test_accuracy = ((test_pred > 0.5).float() == y_test_tensor).float().mean().item()
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    # حفظ النموذج
    torch.save(model.state_dict(), "model.pth")
    return model, test_accuracy, test_loss.item()

train_model({'learning_rate': 0.001, 'hidden_dim': 32, 'dropout_rate': 0.2})     


Epoch 1/20, Loss: 0.6686
Epoch 2/20, Loss: 0.5616
Epoch 3/20, Loss: 0.4686
Epoch 4/20, Loss: 0.3957
Epoch 5/20, Loss: 0.3300
Epoch 6/20, Loss: 0.3056
Epoch 7/20, Loss: 0.2655
Epoch 8/20, Loss: 0.2568
Epoch 9/20, Loss: 0.2428
Epoch 10/20, Loss: 0.2335
Epoch 11/20, Loss: 0.2219
Epoch 12/20, Loss: 0.2163
Epoch 13/20, Loss: 0.2178
Epoch 14/20, Loss: 0.2159
Epoch 15/20, Loss: 0.2118
Epoch 16/20, Loss: 0.2087
Epoch 17/20, Loss: 0.2098
Epoch 18/20, Loss: 0.2103
Epoch 19/20, Loss: 0.2039
Epoch 20/20, Loss: 0.2005
Test Loss: 0.3070, Test Accuracy: 0.8783


(SimpleNN(
   (net): Sequential(
     (0): Linear(in_features=9, out_features=32, bias=True)
     (1): ReLU()
     (2): Dropout(p=0.2, inplace=False)
     (3): Linear(in_features=32, out_features=1, bias=True)
     (4): Sigmoid()
   )
 ),
 0.8783068656921387,
 0.30697834491729736)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [9]:
 import mlflow
 import mlflow.pytorch
 def objective(trail):
    params = {
        'learning_rate': trail.suggest_float('learning_rate', 1e-5, 1e-2, log=True),
        'hidden_dim': trail.suggest_int('hidden_dim', 16, 128),
        'dropout_rate': trail.suggest_float('dropout_rate', 0.0, 0.5)
    }
    with mlflow.start_run():
        model, accuracy, loss = train_model(params)
        mlflow.log_params(params)
        mlflow.log_metric('accuracy', accuracy)
        mlflow.log_metric('loss', loss)
        mlflow.pytorch.log_model(model, "model")
    
    
    return {'loss': loss, 'accuracy': accuracy} 

In [10]:
import optuna
mlflow.set_experiment("Machine_Failure_Prediction_Experiment")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2025-06-18 20:11:40,328] A new study created in memory with name: no-name-e097d693-4e04-4a52-b1a4-a11ca73b1da5


Epoch 1/20, Loss: 0.3401
Epoch 2/20, Loss: 0.2294
Epoch 3/20, Loss: 0.2098
Epoch 4/20, Loss: 0.2082
Epoch 5/20, Loss: 0.2040
Epoch 6/20, Loss: 0.1780
Epoch 7/20, Loss: 0.1817
Epoch 8/20, Loss: 0.1813
Epoch 9/20, Loss: 0.1698
Epoch 10/20, Loss: 0.1746
Epoch 11/20, Loss: 0.1744
Epoch 12/20, Loss: 0.1524
Epoch 13/20, Loss: 0.1605
Epoch 14/20, Loss: 0.1460
Epoch 15/20, Loss: 0.1638
Epoch 16/20, Loss: 0.1471
Epoch 17/20, Loss: 0.1481
Epoch 18/20, Loss: 0.1545
Epoch 19/20, Loss: 0.1313
Epoch 20/20, Loss: 0.1341
Test Loss: 0.4085, Test Accuracy: 0.8783


2025/06/18 20:13:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-06-18 20:13:02,030] Trial 0 failed with parameters: {'learning_rate': 0.007785722973294019, 'hidden_dim': 127, 'dropout_rate': 0.3135822836051216} because of the following error: The value {'loss': 0.4084722101688385, 'accuracy': 0.8783068656921387} could not be cast to float.
[W 2025-06-18 20:13:02,030] Trial 0 failed with value {'loss': 0.4084722101688385, 'accuracy': 0.8783068656921387}.


Epoch 1/20, Loss: 0.3318
Epoch 2/20, Loss: 0.2151
Epoch 3/20, Loss: 0.2004
Epoch 4/20, Loss: 0.1870
Epoch 5/20, Loss: 0.1790
Epoch 6/20, Loss: 0.1809
Epoch 7/20, Loss: 0.1689
Epoch 8/20, Loss: 0.1610
Epoch 9/20, Loss: 0.1496
Epoch 10/20, Loss: 0.1543
Epoch 11/20, Loss: 0.1441
Epoch 12/20, Loss: 0.1443
Epoch 13/20, Loss: 0.1379
Epoch 14/20, Loss: 0.1414
Epoch 15/20, Loss: 0.1420
Epoch 16/20, Loss: 0.1284
Epoch 17/20, Loss: 0.1271
Epoch 18/20, Loss: 0.1368
Epoch 19/20, Loss: 0.1299
Epoch 20/20, Loss: 0.1112
Test Loss: 0.4613, Test Accuracy: 0.8730


2025/06/18 20:13:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-06-18 20:13:13,537] Trial 1 failed with parameters: {'learning_rate': 0.0064190295187403585, 'hidden_dim': 107, 'dropout_rate': 0.09362316126493303} because of the following error: The value {'loss': 0.46129778027534485, 'accuracy': 0.8730158805847168} could not be cast to float.
[W 2025-06-18 20:13:13,546] Trial 1 failed with value {'loss': 0.46129778027534485, 'accuracy': 0.8730158805847168}.


...

2025/06/18 20:16:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-06-18 20:16:04,289] Trial 17 failed with parameters: {'learning_rate': 2.249155292196195e-05, 'hidden_dim': 94, 'dropout_rate': 0.13703770927208175} because of the following error: The value {'loss': 0.6453174948692322, 'accuracy': 0.6772486567497253} could not be cast to float.
[W 2025-06-18 20:16:04,289] Trial 17 failed with value {'loss': 0.6453174948692322, 'accuracy': 0.6772486567497253}.


Epoch 1/20, Loss: 0.6308
Epoch 2/20, Loss: 0.5687
Epoch 3/20, Loss: 0.4988
Epoch 4/20, Loss: 0.4270
Epoch 5/20, Loss: 0.3617
Epoch 6/20, Loss: 0.3075
Epoch 7/20, Loss: 0.2720
Epoch 8/20, Loss: 0.2490
Epoch 9/20, Loss: 0.2359
Epoch 10/20, Loss: 0.2286
Epoch 11/20, Loss: 0.2245
Epoch 12/20, Loss: 0.2083
Epoch 13/20, Loss: 0.2163
Epoch 14/20, Loss: 0.2162
Epoch 15/20, Loss: 0.2079
Epoch 16/20, Loss: 0.2038
Epoch 17/20, Loss: 0.2007
Epoch 18/20, Loss: 0.1988
Epoch 19/20, Loss: 0.1988
Epoch 20/20, Loss: 0.2021
Test Loss: 0.3067, Test Accuracy: 0.8571


2025/06/18 20:16:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-06-18 20:16:15,632] Trial 18 failed with parameters: {'learning_rate': 0.0010574809551475454, 'hidden_dim': 17, 'dropout_rate': 0.08369094523444742} because of the following error: The value {'loss': 0.306656152009964, 'accuracy': 0.8571428656578064} could not be cast to float.
[W 2025-06-18 20:16:15,632] Trial 18 failed with value {'loss': 0.306656152009964, 'accuracy': 0.8571428656578064}.


Epoch 1/20, Loss: 0.6846
Epoch 2/20, Loss: 0.6561
Epoch 3/20, Loss: 0.6510
Epoch 4/20, Loss: 0.6300
Epoch 5/20, Loss: 0.6125
Epoch 6/20, Loss: 0.6122
Epoch 7/20, Loss: 0.6022
Epoch 8/20, Loss: 0.5856
Epoch 9/20, Loss: 0.5770
Epoch 10/20, Loss: 0.5599
Epoch 11/20, Loss: 0.5485
Epoch 12/20, Loss: 0.5358
Epoch 13/20, Loss: 0.5306
Epoch 14/20, Loss: 0.5205
Epoch 15/20, Loss: 0.5079
Epoch 16/20, Loss: 0.4972
Epoch 17/20, Loss: 0.4852
Epoch 18/20, Loss: 0.4732
Epoch 19/20, Loss: 0.4550
Epoch 20/20, Loss: 0.4545
Test Loss: 0.4692, Test Accuracy: 0.8519


2025/06/18 20:16:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-06-18 20:16:25,752] Trial 19 failed with parameters: {'learning_rate': 0.00015673159963126723, 'hidden_dim': 29, 'dropout_rate': 0.22440947833075836} because of the following error: The value {'loss': 0.46916496753692627, 'accuracy': 0.8518518805503845} could not be cast to float.
[W 2025-06-18 20:16:25,752] Trial 19 failed with value {'loss': 0.46916496753692627, 'accuracy': 0.8518518805503845}.
